<div class="cite2c-biblio"></div><div class="cite2c-biblio"></div># Topic Modeling 

In this notebook we wiil texplain what we first understand by topic modelling, what are the most used methods for topic modelling and I will conclude with dynamic topic modeling.

### Topic Modelling Definition

### Topic Modelling Methods

### Dynamic Topic Modelling

In standard topic modelling we assume the order of document does not matter and therfore this approach is not suitable for time-stamped data. 
Dynamic topic modelling on the other end approach to track how languages and topic changes in a time-stamped corpus.

_cite this paper from Derek Greene here_

### Approach for the topic modelling 

The proposed approach is a two level one 

the first step is to divide the corpus into n time windows of equals durations for example days, hours, and any other time windows we want 

- On the first we apply the topic modelling techic to the document in each time windows to produce windows topics. As per the topic modelling definition the result of this step generate n windowfs topics matrix and each windows we have 2 differents matrix one is the document term matrix $W$ and the other one is the topic term matrix $H$.


- On the second  level  we apply another MNF to all topic from step one to find dynamic topic which span multiple times windows. In this step we bacically take the most important topic in each $H$ matrix form step one and stack them row wise to have another topic per time windows and term matrix which we consider as an input matrix after another NMF decomposition of this matrix we have the time topic matrix and another topic term matrix.

The dynaminc algorithm can be represented in the fololwing steps : 

- we start with an empty matrix $B$ 
- for each windows topic model $M_t$: 
    - for each row in the $M_t$ matrix select $n$ top - ranked topic term and fill the other row with zeroes
    - add the vector as a new vector of $B$
- Once all the row have been added removed all the colums with zeros everywhere

The $B$ resulted from this operation is an $i$ x $m'$ matrix where  i is the number of all the topic documents and m' is the number of most used terms in those topics . from the last step of our algo we can see that $m' <<< m $ with this we can see that we have removed all the terms which are not most used in our timeline

#### Analysis of the B matrix 

We all know that the row of the $B$ matrix are the time windows topic and the columns are the top terms in each time windows topic.

Having the new $B$ matrix we can now apply another $NMF$  factorization to get the evolution of the dynamic topic over the time. 

He factorization will of B will yield the following result $B = UV$ , in this decompozition we have 2 matrixes $U$ and $V$.

What do we have in those matrixes ? 

$U$ : the row of this matrix are the time windows topic, and the colums we have the weight of the of each dynamic topic.

$V$ : the values in the row are the dynamic topic themselves and the colums are the terms describing the dynamic topics and their weights 

We can use the weight of the dynamics topic in $U$ to track the evolution of topic over time in our document we can find when it appear and when it when it disapear in the corpus 

Now let move to the application of those methods and see what result we will get .

##### Coding the dynamic topic modelling

#### Loading the data

In the previous notebook about data cleanning we saved our cleanned dataset under `../data/cleanned_tweets_2021.csv`

Here we will just load it and apply a tfid vectorizer to it to get our documend term matrix that we will split into the different time windows 

In [1]:
import pandas as pd 
import numpy as np 

/Users/es.py/Projects/Personal/nlp_course_coursera_russia_uni/.venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = pd.read_csv('../data/cleanned_tweets_2021.csv', index_col='id', parse_dates=['created_at'])[['created_at', 'cleanned_text']]

/Users/es.py/Projects/Personal/nlp_course_coursera_russia_uni/.venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Step one split the data into time windows 

In [9]:
min_date = data.created_at.min().strftime('%d-%b-%Y')

In [10]:
max_date = data.created_at.max().strftime('%d-%b-%Y')

From this we can see that our data where collected from {{min_date}} to {{max_date}}

let us now split it into differend time windows of different using the week as the time measurement 

In [20]:
windows_groups = data.groupby(pd.Grouper(key="created_at", freq="1W"))

In [136]:
windows_data = [df for time, df in windows_groups]

In [137]:
len(windows_data)

32

In [138]:
windows_data[-10]

,created_at,cleanned_text
id,,
1315442075488473093,2020-10-12 00:00:05,fall offcolored kirby get digitalreservetoken
1315442081998004224,2020-10-12 00:00:06,vouloir venir culpabiliser traiter xénophobe p...
1315442083612819456,2020-10-12 00:00:07,cinéma premier projection film lagence présenc...
1315442148964274176,2020-10-12 00:00:22,tête rien
1315442210092048384,2020-10-12 00:00:37,learn mna racial diversity committ statement s...
...,...,...
1317978565154119680,2020-10-18 23:59:11,manooo evento amo
1317978669252489216,2020-10-18 23:59:36,started going
1317978686210048006,2020-10-18 23:59:40,bruno miteyo nyeng nouveau chef maison civil c...


After splitting we can find that our dataset is splited in time windows of {{len(windows_data)}} weeks , with the splitting done we can continue with the topic modelling

Okay what is next is to start analyzing the topic in each time windows and build a pipeline to cobine the results

Should I build the vectorizer for all the topic or for each time windows I need to build  a tokenizer ? 

Dereek suggest to apply the preproccessing for each time windows , we are going to build the documend term matrix for every timewindows and return the document term matrix for each time windows.

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [140]:
vectorizer = TfidfVectorizer(lowercase=True, 
                             strip_accents="unicode", 
                             ngram_range = (1, 2)) 

In [142]:
windows_data[1]

,created_at,cleanned_text
id,,


In [143]:
windows_data = list(filter( lambda x: not x.empty, windows_data))

In [144]:
len(windows_data)

15

In [152]:
windows_data[2].head(49).iloc[1].cleanned_text

'présidence cabinet chef cabinet conseiller spécial chef etat matir'

In [155]:
doc_term_matrix = vectorizer.fit_transform(windows_data[0].cleanned_text.dropna())

In [72]:
doc_term_matrix.shape

(14561, 53641)

In [73]:
terms = vectorizer.get_feature_names()

we have the for the first time windows the a tdifdf matrix of {{doc_term_matrix.shape[0]}} rows and {{doc_term_matrix.shape[1]}} colunms, those columns are the number of words and bi_grams we have in our corpus

We can do futher preprocessing by looking into the most common ngrams we have in our corpus by using the following function 

In [74]:
from operator import itemgetter
def rank_terms( doc_term_matrix, terms ):
    # get the sums over each column
    sums = doc_term_matrix.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=itemgetter(1), reverse=True)

In [92]:
rank_terms(doc_term_matrix, terms)[:30]

[('thread', 218.83351691982568),
 ('zair', 217.28855207461424),
 ('anecdote', 214.5299626363258),
 ('anecdote zair', 214.5299626363258),
 ('thread anecdote', 214.5299626363258),
 ('covid', 96.90122318562476),
 ('mai', 91.11460570791263),
 ('pays', 81.31555017712598),
 ('jour', 72.42982024090871),
 ('ministre', 61.22501661962492),
 ('cas', 60.341136153752345),
 ('tshisekedi', 59.57607977234457),
 ('kabila', 56.10635901834737),
 ('kinshasa', 55.67061650966388),
 ('general', 54.4249999633158),
 ('president', 54.33813823016058),
 ('dire', 48.45230863613515),
 ('felix', 47.58595614377553),
 ('national', 46.989097460364775),
 ('demande', 46.66212030084455),
 ('passeport', 46.48247484132089),
 ('arreter', 46.340508559153186),
 ('felix tshisekedi', 45.19220772334301),
 ('justice', 44.06933342729221),
 ('creer', 43.74798354294154),
 ('pouvoir', 43.652306778435666),
 ('passer', 41.289866489392864),
 ('nouveau', 40.89890882983724),
 ('monde', 40.14428955122549),
 ('jeune', 39.82796088655963)]

Here we have the twenty most recent terms in the first time windows , let us know apply the topic modelling to it to see how what will result from it

In [79]:
k = 10

let us start with a k = {{k}} and then we will incorporate modele selection and performance measurenment metrics later 

Non svd is recomend to because it provide more reliable results (cite: https://www.sciencedirect.com/science/article/abs/pii/S0031320307004359?via%3Dihub)


In [80]:
from sklearn.decomposition import NMF
model = NMF(n_components=k, init="nndsvd", random_state=0)

In [82]:
W = model.fit_transform(doc_term_matrix)
H = model.components_

In [85]:
W.shape

(14561, 10)

In [91]:
W[3,:].round(3)

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.014, 0.017,
       0.   ])

In [86]:
H.shape

(10, 53641)

H contains the term weight for all topic and in from that we can pick one term and check how it is weigthed in each topic 

In [94]:
term_index = terms.index('passeport')
H[:, term_index].round(3)

array([0.   , 0.   , 0.006, 0.004, 0.   , 0.013, 0.003, 0.   , 0.009,
       0.002])

In this we can find that the term passeport is highly linked with approximately 3 topics

Let see if now our topic make senses , we will use a functin called topic descriptor , for each topic it will return the top most ranked term in it.

In [99]:
def get_descriptor( terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( (terms[term_index], H[topic_index, term_index].round(2)) )
    return top_terms

In [105]:
descriptors = []
for topic_index in range(k):
    descriptor = get_descriptor( terms, H, topic_index, 10 ) 
    descriptor = [f"{x[0]}=:{x[1]}" for x in descriptor]
    descriptor = ', '.join(descriptor)
    current_topic = topic_index+1
    print(f"Topic {current_topic:.1f} === describe by : {descriptor}")
    print('=:'*20)

Topic 1.0 === descripe by : thread=:2.098826947152738, anecdote zair=:2.09821705004985, thread anecdote=:2.09821705004985, anecdote=:2.09821705004985, zair=:2.062225949589958, symboliser envahissement=:0.005933999451435938, malheur boot=:0.005933999451435938, symboliser=:0.005933999451435938, boot=:0.005933999451435938, jour symboliser=:0.005933999451435938
=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:
Topic 2.0 === descripe by : world shortest=:0.7977587288636605, shortest set=:0.7977587288636605, leav villag=:0.7977587288636605, set people=:0.7977587288636605, african pygmie=:0.7977587288636605, people friendly=:0.7977587288636605, pygmie world=:0.7977587288636605, never leav=:0.7977587288636605, friendly=:0.7977587288636605, shortest=:0.7977587288636605
=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:=:
Topic 3.0 === descripe by : rappeur damso=:0.8278004304520612, fondation afin=:0.8278004304520612, vol ressource=:0.8278004304520612, damso=:0.8278004304520612, afin lutter=:0.8278004304520612, dam

From a human evaluation percepective the topic and the description make sense we can now see how the. enmbedding train in our corpus make sense in for those

### Evaluation of our models and parameter selection 

In this we will use coherence measures using embeddings to detects how good are our generated topics. 

what are those coherence measures? 

Here we will define the topice coherence which the sum of pair wise similaries of words that describes a topic? _why not weigthed it by topic weigth?_

Basically it is define as 


For each topic we can do :

$TC =  \sum_{j=2}^{N}\sum_{i=2}^{j-1}{similarity(w_i, w_j)}$


For all the topics we define it as : 

it is the mean of TC for all the topic in the $H$ matrix 

The similarity measure we use is the cosine distance between the enmbeddings of words we have in our corpus. We can use any enmbedding techinc but here we suggest to learn embedding for our corpus since we have many term specific to the congolese context which are not in the well know enmbeddings 

#### Learning words embeddings

We are planning to use pmi mutual information to train our word embeddings , and we evaluate the results and how they perform for mostly misspeled words if they are not performing well we will try to train a custom character level embeddings

We need to check [this](https://towardsdatascience.com/embedding-for-spelling-correction-92c93f835d79) to check how we can improve our embeddings to take into consideration misspled words 

Cite the facebook paper where they build a model to check for misspled charactere

This one : __*arXiv:1905.09755v1 [cs.CL] 23 May 2019*__

continue wih this research to complete the embedding trainning 

https://www.kaggle.com/alexklibisz/simple-word-vectors-with-co-occurrence-pmi-and-svd

We are planning to use to train the charactere n-gram embedding

@article{bojanowski2017enriching,
  title={Enriching Word Vectors with Subword Information},
  author={Bojanowski, Piotr and Grave, Edouard and Joulin, Armand and Mikolov, Tomas},
  journal={Transactions of the Association for Computational Linguistics},
  volume={5},
  year={2017},
  issn={2307-387X},
  pages={135--146}
}